In [1]:
import mysql.connector
import pandas as pd
import numpy as np

In [2]:
i_cols = ['movie_id', 'movie_title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('data/ml-100k/u.item', sep='|', names=i_cols,
 encoding='latin-1')

items.drop(['video_release_date'],axis=1,inplace=True)

n_items = items.shape[0]
print('Number of items:', n_items)
items.head()

Number of items: 1682


,movie_id,movie_title,release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,Comedy,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="recommendation"
)
mycursor = mydb.cursor()
print(mycursor)

CMySQLCursor: (Nothing executed yet)


In [4]:
def insert(data):
    sql1 = "INSERT INTO items (movie_id, movie_title) VALUES (%s, %s)"
    val1 = (data['movie_id'], data['movie_title'])
    mycursor.execute(sql1, val1)
    types = ['unknown', 'Action', 'Adventure',
             'Animation', 'Children_s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
             'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']
    sql2 = "INSERT INTO item_type (movie_id, movie_type) VALUES (%s, %s)"
    val2 = [(data['movie_id'], _type) for _type in types if data[_type]]
    mycursor.executemany(sql2, val2)

In [5]:
items.apply(insert,axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1677    None
1678    None
1679    None
1680    None
1681    None
Length: 1682, dtype: object

In [6]:
mydb.commit()

In [5]:
u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('data/ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

In [6]:
users.drop(['zip_code'],axis=1,inplace=True)

In [7]:
users.head()

,user_id,age,sex,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


In [8]:
def insertUser(data):
    sql = "INSERT INTO users (user_id, age, sex, occupation) VALUES (%s, %s, %s, %s)"
    val = (data['user_id'],data['age'],data['sex'],data['occupation'])
    mycursor.execute(sql, val)

In [9]:
mycursor = mydb.cursor()
users.apply(insertUser,axis=1)
mydb.commit()

In [20]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings = pd.read_csv('data/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')

In [21]:
ratings.head()
ratings.drop(['unix_timestamp'],axis=1,inplace=True)

In [22]:
ratings.head()

,user_id,movie_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [23]:
def insertRating(data):
    sql = "INSERT INTO ratings (user_id, movie_id, rating) VALUES (%s, %s, %s)"
    val = (int(data['user_id']),int(data['movie_id']),int(data['rating']))
    mycursor.execute(sql, val)

In [24]:
mycursor = mydb.cursor()
ratings.apply(insertRating,axis=1)
mydb.commit()